In [2]:
!pip install dfply
import pandas as pd
import numpy as np
from datetime import datetime
import datetime as dt
from dateutil import parser
import re
import math
from dfply import *
import matplotlib.pyplot as plt
import seaborn as sns
from dateutil.relativedelta import relativedelta
from datetime import date

     |████████████████████████████████| 614kB 2.7MB/s 


/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [0]:
def years_left(date_1):
  result = math.nan
  try:
    result = relativedelta(date_1, date.today()).years
  except:
    pass
  return result

def get_age(date_1):
  result = math.nan
  try:
    result = relativedelta(date.today(),date_1).years
  except:
    pass
  return result

def get_current_mv(tm_Id):
  global markval
  result = math.nan
  try:
    mv_2020 = markval['2020-06-30']
    current_value = mv_2020[mv_2020.tm_Id == tm_Id].head(1)["MV"][0]
    result = current_value
  except:
    pass
  return result
  
def get_main_position(position):
  strikers = ['CF', 'ST', 'LS', 'RS', 'FW']
  wingers = ['LW', 'RW', 'LF', 'RF']
  clean_midfielders = ['CM', 'LCM', 'RCM','LM', 'RM', 'MF']
  def_midfielders = ['CDM','RDM', 'LDM']
  att_midfielders = ['CAM', 'RAM', 'LAM']
  full_backs = ['LB', 'RB', 'LWB', 'RWB']
  defenders = ['CB', 'LCB', 'RCB', 'DF']
  goalkeepers = ['GK']
  result = ""
  if(position in strikers):
    result = "attackers"
  elif(position in wingers):
    result = "attackers"
  elif(position in clean_midfielders):
    result = "midfielders"  
  elif(position in def_midfielders):
    result = "midfielders"
  elif(position in att_midfielders):
    result = "midfielders"
  elif(position in full_backs):
    result = "defenders"
  elif(position in defenders):
    result = "defenders"
  elif(position in goalkeepers):
    result = "goalkeepers"
  return result

def get_league(team):
  result = None
  global teams_leagues
  try:
    tl_2020 = teams_leagues['2020-06-30']
    team = tl_2020[tl_2020.Club == team]
    league = team.League
    result = league
  except:
    pass
  return result

def get_country(league):
  global countries_leagues
  result = None
  try:
    league = countries_leagues[countries_leagues.league == league]
    result = league.country.unique()[0]
  except:
    pass
  return result

In [0]:
players = pd.read_pickle("/content/drive/My Drive/Capstone/Data/Clean/Players_Clean.pkl")
teams_leagues = pd.read_pickle("/content/drive/My Drive/Capstone/Data/Clean/teams_leagues_Clean.pkl")
countries_leagues = pd.read_pickle("/content/drive/My Drive/Capstone/Data/Clean/countries_leagues_Clean.pkl")
markval = pd.read_pickle("/content/drive/My Drive/Capstone/Data/Clean/markval_Clean.pkl")

In [0]:
continents = pd.read_pickle("/content/drive/My Drive/Capstone/Data/Clean/continents.pkl")

[EU, NA, SA, AS, AF, NOCONT, OC]
Categories (7, object): [EU, NA, SA, AS, AF, NOCONT, OC]

In [0]:
players["years_left"] = players.Contract_Expires.apply(years_left) 
players["age"] = players.DOB.apply(get_age) 

In [0]:
players = players[players.Height >= 150]

In [0]:
players["current_mv"] = players.tm_Id.apply(get_current_mv)

In [0]:
players["main_position"] = players.Position.apply(get_main_position)

In [0]:
players["main_position"] = players["main_position"].astype("category")

In [0]:
players["league"] = players["Team"].apply(get_league)

In [0]:
teams_leagues["Country"] = teams_leagues.League.apply(get_country)

In [0]:
teams_leagues = teams_leagues[teams_leagues.Country.notnull()]
teams_leagues.to_pickle("/content/drive/My Drive/Capstone/Data/Clean/teams_leagues_Clean.pkl")

In [0]:
countries_unique = list(countries_leagues.country.unique())
countries_unique.append("Serbia")
countries_unique.append("Bermuda")
countries_unique.append("Kosovo")
countries_unique.append("Montenegro")
players["nationalities"] = players.Nationality.apply(lambda nat: list(filter(lambda x : x in nat.strip(), countries_unique)))

In [0]:
def get_nationality(row):
  nats = row.nationalities
  nat = row.Nationality
  if len(nats) >= 1:
    return str(nats[0]).strip()
  else:
    return str(nat).strip()

players["Nationality"] = players.apply(get_nationality, axis = 1)

In [12]:
players["continent"].unique()

[EU, NA, SA, AS, AF, NOCONT, OC]
Categories (7, object): [EU, NA, SA, AS, AF, NOCONT, OC]

In [0]:
def get_continent(nat):
  global continents
  result = "NOCONT"
  try:
    country_cont = continents[continents.country == nat]
    result = country_cont["continent"].values[0]
  except Exception as e:
    print(str(e))
    pass
  print(result)
  return result

players["continent"] = players["nationality"].apply(get_continent)

In [0]:
players.continent.unique()
players.continent = players.continent.astype("category")
players.continent.unique()
# players.dtypes
# players.to_pickle("/content/drive/My Drive/Capstone/Data/Clean/Players_Clean.pkl")

In [0]:
df = players
df.columns = [col.lower() for col in df.columns]
df.columns = [col.replace('team', 'club') for col in df.columns]
df.columns = [col.replace('position', 'field_position') for col in df.columns]

In [0]:
df.continent = df.continent.astype("category")

In [0]:
df.to_pickle("/content/drive/My Drive/Capstone/Data/Clean/Players_Clean.pkl")

In [0]:
df_1 = pd.read_pickle("/content/drive/My Drive/Capstone/Data/Clean/Players_Clean.pkl")

In [31]:
df_1.dtypes

tm_id                           int64
name                           object
club                           object
nationality                    object
dob                    datetime64[ns]
height                        float64
sf                           category
field_position               category
joined                 datetime64[ns]
contract_expires       datetime64[ns]
followers                     float64
sofifa_id                     float64
years_left                   category
age                           float64
current_mv                    float64
main_field_position          category
league                         object
nationalities                  object
continent                    category
years_at_club                 float64
dtype: object

In [22]:
df_1.continent.unique()

[EU, NA, SA, AS, AF, NOCONT, OC]
Categories (7, object): [EU, NA, SA, AS, AF, NOCONT, OC]

In [0]:
df_1["years_at_club"] = df_1["joined"].apply(get_age)

In [0]:
df_1.to_pickle("/content/drive/My Drive/Capstone/Data/Clean/Players_Clean.pkl")